# Practical Exam: Supermarket Loyalty

International Essentials is an international supermarket chain.

Shoppers at their supermarkets can sign up for a loyalty program that provides rewards each year to customers based on their spending. The more you spend the bigger the rewards. 

The supermarket would like to be able to predict the likely amount customers in the program will spend, so they can estimate the cost of the rewards. 

This will help them to predict the likely profit at the end of the year.

## Data

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|


# Task 1

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "loyalty.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|

In [1]:
# Use this cell to write your code for Task 1
import pandas as pd 
import numpy as np

filepath = "loyalty.csv"
loyalty_df = pd.read_csv(filepath)

# Make sure there is no negative spend
assert len(loyalty_df[loyalty_df['spend'] < 0]['spend']) == 0, 'Negative Spend Detected!'
assert loyalty_df['spend'].isna().sum() == 0, 'Null Values found for Spend! Needs Addressing!'
# If for some reason, we missed something
loyalty_df['spend'].fillna(0,inplace=True)

# Cleaning First Month 
loyalty_df.loc[loyalty_df['first_month'] == ".",'first_month'] = 0
assert len(loyalty_df[loyalty_df['first_month'] == "."]) == 0, 'Some NaN Values were missed'
loyalty_df['first_month'] = loyalty_df['first_month'].astype('float64')

# Making sure that the min value is greater than 0
assert loyalty_df['first_month'].min() >= 0, 'Some Values are Negative'
assert loyalty_df['first_month'].isna().sum() == 0, 'Null Values found for First Month! Needs Addressing'

# Cleaning Items in First Month 
assert loyalty_df['items_in_first_month'].dtype == 'int64', 'Items in First Month is in the Wrong Data Type'
assert len(loyalty_df[loyalty_df['items_in_first_month']<=0]) == 0, 'Error! Negative `Items in First Month`'
assert loyalty_df['items_in_first_month'].isna().sum() == 0, "Null Values found in `Items in First Month`"
loyalty_df['items_in_first_month'].fillna(0, inplace=True)

# Cleaning Regions Data 
loyalty_df['region'] = loyalty_df['region'].astype('category')
regions = ['Americas', 'Asia/Pacific', 'Europe', 'Middle East/Africa']
assert len(loyalty_df[loyalty_df['region'].isin(regions)]) == len(loyalty_df['region']),'Values Found Outside Specified Regions'
assert loyalty_df['region'].isna().sum() == 0, 'Null Values found in Region'
# In case testing code introduces missing values
loyalty_df['region'].fillna('Unknown',inplace=True)

# Cleaning Loyalty Years
loyalty_df['loyalty_years'] = loyalty_df['loyalty_years'].astype('O')
loyalty_df['loyalty_years'].unique()
# Creating ordered categories
categories = ['0-1', '1-3', '3-5', '5-10', '10+']
loyalty_df['loyalty_years'] = pd.Categorical(loyalty_df['loyalty_years'],categories=categories,ordered=True)
assert len(loyalty_df[~loyalty_df['loyalty_years'].isin(categories)]) == 0, 'Values Found Outside Loyalty Years Category'
assert loyalty_df['loyalty_years'].isna().sum() == 0, 'Null Values found in Loyalty Years'
loyalty_df['loyalty_years'].fillna('0-1',inplace=True)

# Cleaning Joining Month 
loyalty_df['joining_month'].fillna('Unknown',inplace=True)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Unknown']
assert len(loyalty_df[loyalty_df['joining_month'].isin(months)]) == len(loyalty_df['joining_month']), 'Values Found Outside Joining Months'
assert loyalty_df['joining_month'].isna().sum() == 0, 'Values Found Outside Joining Months'
loyalty_df['joining_month'] = loyalty_df['joining_month'].astype('category')

# Cleaning Promotion 
loyalty_df['promotion'] = loyalty_df['promotion'].str.title()
loyalty_df['promotion'] = loyalty_df['promotion'].astype('category')
assert loyalty_df['promotion'].isna().sum() == 0, 'Null Values found in Promotion'

clean_data = loyalty_df.copy()

# Task 2 

The team at International Essentials have told you that they have always believed that the number of years in the loyalty scheme is the biggest driver of spend. 

Producing a table showing the difference in the average spend by number of years in the loyalty programme along with the variance to investigate this question for the team.

 - You should start with the data in the file 'loyalty.csv'.

 - Your output should be a data frame named `spend_by_years`. 

 - It should include the three columns `loyalty_years`, `avg_spend`, `var_spend`. 

 - Your answers should be rounded to 2 decimal places.   

In [2]:
# Use this cell to write your code for Task 2
loyalty_df = pd.read_csv('loyalty.csv')
spend_by_years = loyalty_df.groupby('loyalty_years')['spend'].agg(['mean','var']).sort_values(by='loyalty_years').round(2).reset_index()
spend_by_years.rename(columns={'mean':'avg_spend','var':'var_spend'},inplace=True)
print(spend_by_years)

  loyalty_years  avg_spend  var_spend
0           0-1     110.56       9.30
1           1-3     129.31       9.65
2           10+     117.41      16.72
3           3-5     124.55      11.09
4          5-10     135.15      14.10


# Task 3

Fit a baseline model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [3]:
from sklearn.linear_model import LinearRegression

# Task 3 - Base Model
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Defining the categorical columns
cat_cols = ['promotion', 'joining_month', 'region', 'loyalty_years']

# Process training data
train_data[cat_cols] = train_data[cat_cols].astype('category')
train_data = pd.get_dummies(train_data, drop_first=True)

# Process test data
test_data[cat_cols] = test_data[cat_cols].astype('category')
test_data = pd.get_dummies(test_data, drop_first=True)

# Train base model
X_train = train_data.drop('spend', axis=1).values
y_train = train_data['spend'].values
base_model = LinearRegression()
base_model.fit(X_train, y_train)

# Make predictions
X_test = test_data.values
base_model_predictions = base_model.predict(X_test)

# Create final base result DataFrame
base_result = pd.DataFrame({
    'customer_id': test_data['customer_id'],
    'spend': base_model_predictions.round(2)  
})

# Task 4

Fit a comparison model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [11]:
# Use this cell to write your code for Task 4
from sklearn.ensemble import GradientBoostingRegressor

# Using GBR for predictions
compare_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42
)
compare_model.fit(X_train, y_train)

# Storing the predictions
compare_model_results = compare_model.predict(X_test)

assert compare_model_results.shape[0] == test_data.shape[0], 'Shape Mismatch between Prediction and Test Files'

# Make predictions
compare_model_results = compare_model.predict(X_test)

# Create final compare result DataFrame
compare_result = pd.DataFrame({
    'customer_id': test_data['customer_id'],
    'spend': compare_model_results.round(2)
})

compare_result

,customer_id,spend
0,5,140.84
1,7,148.86
2,16,139.79
3,17,151.05
4,19,153.64
...,...,...
245,1216,134.92
246,1225,148.77
247,1231,136.87
248,1242,129.87
